In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
from tqdm import tqdm_notebook as tqdm
from selenium import webdriver
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
import warnings
warnings.filterwarnings("ignore")
import os, shutil

import headers

In [2]:
def get_page_num(page_s):
    is_this_a_first_try = 1
    marker = 0
    success = 0
    while success !=1:
        if is_this_a_first_try ==1:
            page = page_s
            is_this_a_first_try = 0
        else:
            page = next_page
            next_page = 'https://www.cian.ru'+str(data[len(data)-1]).split('href="')[1].split('">..<')[0].replace('amp;','')
        data, m = headers.connection(page,marker)
        marker = m
        data = BeautifulSoup(data, 'lxml').findAll('li', attrs={'class':'_93444fe79c--list-item--2KxXr'})
        if data != []:
            if '>..<' in str(data[len(data)-1]):
                #print("NO!")
                next_page = 'https://www.cian.ru'+str(data[len(data)-1]).split('href="')[1].split('">..<')[0].replace('amp;','')
            else:
                success = 1
                return int(str(data[len(data)-1]).split('</')[-3].split('>')[-1])
        else:
            return -1

def writer(file,a):
    try:
        lines = []
        with open(file, 'r', encoding='UTF-8') as f:
            for el in f:
                line = f.read(el.replace('\n',''))
                lines.append(line)
        f.close()
    except:
        pass
    rez1 = []
    for line in a:
        if line not in lines:
            rez1.append(line)
    with open(file, 'a', encoding='UTF-8') as f:
        for el in rez1:
            f.write(el+'\n')
    f.close()

def note_finder(data):
    title = data.findAll('title')
#     print(title)
#     if deal == 'sale' :    
#         try:
#             notes_num = int(str(title).split('в Москве - ')[1].split('объявл')[0].replace(' ', ''))
#         except:
#             notes_num = 0
#             print('No offers')
#     else:
        # <div class="_93444fe79c--totalOffers--22-FL">
    try:
        notes_num = data.findAll('div', attrs = {'class':'_93444fe79c--totalOffers--22-FL'})
        #print(notes_num)
        notes_num = int(str(notes_num).split('FL">')[1].split(' предлож')[0].replace(' ',''))
    except:
        notes_num = 0
        print('No offers')
    return notes_num
    
def link_extractor(data):
    links = []
    add_links = []
    search_page1 = data.findAll('a', attrs={'class':'c6e8ba5398--header-link--3XZlV'})

    for link in search_page1:
        link = str(link).split()[2].split('"')[1]
        links.append(link)

    search_page2 = data.findAll('a', attrs={'class':'c6e8ba5398--similar--L7_Ac c6e8ba5398--similar--top3--2DuM_'})

    for add_link in search_page2:
        add_links.append('https://www.cian.ru'+str(add_link).split()[3].split('"')[1])

    #print(len(links), len(add_links))

    for add_link in add_links:
        data,m = headers.connection(add_link,0)
        data = BeautifulSoup(data, 'lxml')
        search_page1 = data.findAll('a', attrs={'class':'c6e8ba5398--header-link--3XZlV'})

        for link in search_page1:
            link = str(link).split()[2].split('"')[1]
            if link not in links:
                links.append(link)
    return links

In [3]:
main = '''https://www.cian.ru/cat.php?deal_type={}&engine_version=2&m2=1&offer_type=offices&office_type%5B0%5D={}&p={}&quality=1&region={}&sort=price_square_order'''
page = '''https://www.cian.ru/cat.php?currency=2&deal_type={}&engine_version=2&m2=1&maxprice={}&minprice={}&offer_type=offices&office_type%5B0%5D={}&p={}&quality=1&region={}&sort=price_square_order'''

In [4]:
regions = [4885] #, 1,4885  # 1 - Moscow, 4885  - Nizhniy Novgorod
deal_type = ['sale','rent'] # 'sale','rent'
object_type = [1,2,3,4,5,6,7,9,10,11,12] # 1,2,3,4,5,6,7,9,10,11,12

In [5]:
os.chdir(r'C:\Users\Dmitry\Desktop\PS\ds4_dip\DATA')
original_path = os.getcwd()
original_path

'C:\\Users\\Dmitry\\Desktop\\PS\\ds4_dip\\DATA'

In [6]:
for region in regions:
    for deal in deal_type: # deal_type
        if region == 1:
            if deal == 'sale':
                path = original_path+'\\html\\Moscow\\sale'
            else:
                path = original_path+'\\html\\Moscow\\rent'
        else:
            if deal == 'sale':
                path = original_path+'\\html\\Nizhniy\\sale'
            else:
                path = original_path+'\\html\\Nizhniy\\rent'
        os.chdir(path)
        for i in object_type:
            try:
                os.mkdir('./{i}'.format(i=i))
            except:
                pass

In [9]:
for region in regions:
    print('region = '+str(region))
    for deal in deal_type: # deal_type
        if region == 1:
            if deal == 'sale':
                path = original_path+'\\html\\Moscow\\sale'
            else:
                path = original_path+'\\html\\Moscow\\rent'
        else:
            if deal == 'sale':
                path = original_path+'\\html\\Nizhniy\\sale'
            else:
                path = original_path+'\\html\\Nizhniy\\rent'
        os.chdir(path)
        print('deal_type = '+deal)
        for obj in object_type:
            pp = path+'\\{}'.format(str(obj))
            os.chdir(pp)
            print('object_type = '+str(obj))
            print(os.getcwd())
            marker = 0
            notes = main.format(deal, obj, 1, region)
            data,m = headers.connection(notes,0)
            data = BeautifulSoup(data, 'lxml')
            notes_num = note_finder(data)
            counter = notes_num
            print('Кол-во объявлений = ', notes_num)
            rez = []
            if notes_num <1300:
                # парсим прям тут
                # получаю кол-во страниц
                page_num = get_page_num(notes)
                print("Осталось "+str(page_num)+" страниц и "+str(notes_num)+" объявлений")
                # итерирую по каждой странице
                for p in tqdm(range(1,page_num+1)):
                    # получаю исходник страницы
                    pa = main.format(deal, obj, p,  region)
                    data,m = headers.connection(pa,marker)
                    marker = m
                    # ОБРЕЗКА ПО ДОП. ПРЕДЛОЖЕНИЯМ
                    data = BeautifulSoup(data, 'lxml')
                    if '<h3 class="_93444fe79c--title--299uA">' in str(data):
                        # обрезаю по доп.предложениям
                        ps1 = str(data).split('<h3 class="_93444fe79c--title--299uA">')[0]
                        search_page1 = BeautifulSoup(ps1, 'lxml')
                    else: 
                        search_page1 = data
                    rez += link_extractor(search_page1)
            else:
                
                search_page1 = data
                if deal == 'sale':
                    search_page1 = search_page1.findAll('div', attrs={'class':'c6e8ba5398--column--1rxrN'})
                    d = search_page1[0].findAll('li', attrs={'class':'c6e8ba5398--header-subTerm-item--1pUL4'})
                    e = int(str(d[-1]).split('<div>')[1].split(' руб')[0].replace(' ',''))
                    #print(e)
                else:
                    d = search_page1.findAll('div', attrs={'class':'c6e8ba5398--column--1rxrN'})
                    e = int(str(d[0].findAll('li', attrs={'class':'c6e8ba5398--header-subTerm-item--1pUL4'})[-1]).split('1pUL4">')[1].split(' руб')[0].replace(' ',''))
                minprice = e
                maxprice = e+1000
                print("Самый дешевый вариант стоит ",minprice) 
                #page = '''https://www.cian.ru/cat.php?currency=2&deal_type={}&engine_version=2&m2=1&maxprice={}&minprice={}&offer_type=offices&office_type%5B0%5D={}&p={}&quality=1&region={}&sort=price_square_order'''

                while counter!=0:
                    
                    if counter< 1300:
                        print('Последняя итерация')
                        p = page.format(deal,'',minprice,obj, 1,region)
                        page_num = get_page_num(p)
                        print("Осталось "+str(page_num)+" страниц и "+str(counter)+" объявлений")
                        marker = 0
                        # итерирую по каждой странице
                        for p in tqdm(range(1,page_num+1)):
                            # получаю исходник страницы
                            pa = page.format(deal,"",minprice,obj, p,region)
                            data,m = headers.connection(pa,marker)
                            marker = m
                            # ОБРЕЗКА ПО ДОП. ПРЕДЛОЖЕНИЯМ
                            data = BeautifulSoup(data, 'lxml')
                            if '<h3 class="_93444fe79c--title--299uA">' in str(data):
                                # обрезаю по доп.предложениям
                                ps1 = str(data).split('<h3 class="_93444fe79c--title--299uA">')[0]
                                search_page1 = BeautifulSoup(ps1, 'lxml')
                            else: 
                                search_page1 = data
                            rez += link_extractor(search_page1)
                            counter = 0
                    else:
                        succeed = 0
                        step = int(e*10)
                        val = e
                        sign = '+'
                        print("Ищу оптимальные параметры...")
                        while succeed == 0:
                            p = page.format(deal,maxprice,minprice,obj, 1,region)
                            #print(p)
                            data,m = headers.connection(p,0)
                            data = BeautifulSoup(data, 'lxml')
                            n_nm = note_finder(data)
                            #print(minprice,maxprice, n_nm)
                            if n_nm >= 1300:
                                if n_nm<=1500:
                                    succeed = 1
                                    print('Найдены параметры стоимости для парсинга',minprice,maxprice, n_nm)
                                else:
                                    if step == val:
                                        maxprice-=step
                                        sign = '-'
                                    else:
                                        if sign == '-':
                                            maxprice-=step
                                            sign = '-'
                                        else:
                                            step = int(step/2)
                                            maxprice=maxprice-step
                                            sign = '-'
                            else:
                                if step == val:
                                    maxprice+=step
                                    sign = '+'
                                else:
                                    if sign == '+':
                                        maxprice+=step
                                        sign = '+'
                                    else:
                                        step = int(step/2)
                                        maxprice=maxprice+step
                                        sign = '+'
                        #print(p)
                        counter -= n_nm
                        
                        print("После этого прохода останется еще "+str(counter)+" объявлений")
                        page_num = get_page_num(p)
                        marker = 0
                        # итерирую по каждой странице
                        for p in tqdm(range(1,page_num+1)):
                            # получаю исходник страницы
                            pa = page.format(deal,maxprice,minprice,obj, p,region)
                            data,m = headers.connection(pa,marker)
                            marker = m
                            # ОБРЕЗКА ПО ДОП. ПРЕДЛОЖЕНИЯМ
                            data = BeautifulSoup(data, 'lxml')
                            if '<h3 class="_93444fe79c--title--299uA">' in str(data):
                                # обрезаю по доп.предложениям
                                ps1 = str(data).split('<h3 class="_93444fe79c--title--299uA">')[0]
                                search_page1 = BeautifulSoup(ps1, 'lxml')
                            else: 
                                search_page1 = data
                            rez += link_extractor(search_page1)
                        print('len rez = ', len(rez))
                        minprice = maxprice
            
            print("Записываю в файл...")
            writer('links_to_parce.txt',rez)
            print("Записал!")


region = 4885
deal_type = sale
object_type = 1
C:\Users\Dmitry\Desktop\PS\ds4_dip\DATA\html\Nizhniy\sale\1
Кол-во объявлений =  419
Осталось 15 страниц и 419 объявлений



Записываю в файл...
Записал!
object_type = 2
C:\Users\Dmitry\Desktop\PS\ds4_dip\DATA\html\Nizhniy\sale\2
Кол-во объявлений =  330
Осталось 12 страниц и 330 объявлений



Записываю в файл...
Записал!
object_type = 3
C:\Users\Dmitry\Desktop\PS\ds4_dip\DATA\html\Nizhniy\sale\3
Кол-во объявлений =  52
Осталось 2 страниц и 52 объявлений



Записываю в файл...
Записал!
object_type = 4
C:\Users\Dmitry\Desktop\PS\ds4_dip\DATA\html\Nizhniy\sale\4
Кол-во объявлений =  44
Осталось 2 страниц и 44 объявлений



Записываю в файл...
Записал!
object_type = 5
C:\Users\Dmitry\Desktop\PS\ds4_dip\DATA\html\Nizhniy\sale\5
Кол-во объявлений =  378
Осталось 14 страниц и 378 объявлений


...


Записываю в файл...
Записал!
object_type = 9
C:\Users\Dmitry\Desktop\PS\ds4_dip\DATA\html\Nizhniy\rent\9
Кол-во объявлений =  5
Осталось -1 страниц и 5 объявлений



Записываю в файл...
Записал!
object_type = 10
C:\Users\Dmitry\Desktop\PS\ds4_dip\DATA\html\Nizhniy\rent\10
Кол-во объявлений =  29
Осталось 2 страниц и 29 объявлений



Записываю в файл...
Записал!
object_type = 11
C:\Users\Dmitry\Desktop\PS\ds4_dip\DATA\html\Nizhniy\rent\11
Кол-во объявлений =  34
Осталось 2 страниц и 34 объявлений



Записываю в файл...
Записал!
object_type = 12
C:\Users\Dmitry\Desktop\PS\ds4_dip\DATA\html\Nizhniy\rent\12
Кол-во объявлений =  43
Осталось 2 страниц и 43 объявлений



Записываю в файл...
Записал!
